In [1]:
import requests
import yaml, os, random
from bs4 import BeautifulSoup
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.chat_models import AzureChatOpenAI
from langchain.tools import DuckDuckGoSearchRun, BaseTool
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [2]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ["OPENAI_API_TYPE"] = credentials['AD_OPENAI_API_TYPE']
os.environ["OPENAI_API_VERSION"] = credentials['AD_OPENAI_API_VERSION']
os.environ["OPENAI_API_BASE"] = credentials['AD_OPENAI_API_BASE']
os.environ["OPENAI_API_KEY"] = credentials['AD_OPENAI_API_KEY']

In [3]:
llm = AzureChatOpenAI(
                    deployment_name=credentials['AD_DEPLOYMENT_ID'],
                    model_name=credentials['AD_ENGINE'],
                    temperature=0.9, 
                    max_tokens = 256
                    )

# Standard Tools

In [4]:
search = DuckDuckGoSearchRun()
# defining a single tool
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    )
]

# Custom Tools

In [5]:
def meaning_of_life(input=""):
    return 'The meaning of life is 42 if rounded but is actually 42.17658'

In [6]:
life_tool = Tool(
                name='Meaning of Life',
                func= meaning_of_life,
                description="Useful for when you need to answer questions about the meaning of life. input should be MOL "
                )

In [7]:
def random_num(input=""):
    return random.randint(0,5)

In [8]:
random_tool = Tool(
                    name='Random number',
                    func= random_num,
                    description="Useful for when you need to get a random number. input should be 'random'"
                    )

In [9]:
tools = [
        search, 
        random_tool, 
        life_tool
        ]

# conversational agent memory
memory = ConversationBufferWindowMemory(
                                        memory_key='chat_history',
                                        k=3,
                                        return_messages=True
                                        )


# create our agent
conversational_agent = initialize_agent(
                                        agent='chat-conversational-react-description',
                                        tools=tools,
                                        llm=llm,
                                        verbose=True,
                                        max_iterations=3,
                                        early_stopping_method='generate',
                                        memory=memory
                                        )

In [10]:
conversational_agent("What time is it in London?")



> Entering new AgentExecutor chain...
{
    "action": "duckduckgo_search",
    "action_input": "what time is it in london"
}
Observation: The current local time in London is 14 minutes behind apparent solar time. London on the map. London is the capital of United Kingdom. Latitude: 51.51. Longitude: -0.13; Population: 8,962,000; Open London in Google Maps. Best restaurants in London #1 Al Arez 1 - Lebanese and middle eastern food United Kingdom is not currently observing Daylight Saving Time (DST). London time zone converter to other locations. clear. Time Converters Sun and Moon Articles Timepieces. × . TIME . UNITED-KINGDOM . LONDON . Time now in London, United Kingdom. Constituent Country: England. London. 8:37:42 PM 11/19/2023 GMT +00:00 ... Find out the current local time in London, United Kingdom, and the date and time of daylight saving time (DST) changes in 2023. Compare London time with your local time using the time zone converter and see the location map and population of 

{'input': 'What time is it in London?',
 'chat_history': [],
 'output': 'The current time in London, United Kingdom is 8:37:42 PM on November 19, 2023. London is not currently observing Daylight Saving Time (DST).'}

In [11]:
conversational_agent("Can you give me a random number?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Random number",
    "action_input": "random"
}
```
Observation: 0
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The response to your last comment was a random number, and the number was 0."
}
```

> Finished chain.


{'input': 'Can you give me a random number?',
 'chat_history': [HumanMessage(content='What time is it in London?'),
  AIMessage(content='The current time in London, United Kingdom is 8:37:42 PM on November 19, 2023. London is not currently observing Daylight Saving Time (DST).')],
 'output': 'The response to your last comment was a random number, and the number was 0.'}

In [12]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "The meaning of life is a philosophical and existential question that has been asked by humans for centuries. There is no one definitive answer to this question, and opinions on the topic vary widely depending on cultural, religious, and individual beliefs. Some people believe that the meaning of life is to seek happiness and fulfillment, while others believe it is to achieve spiritual enlightenment or to serve a higher purpose. Ultimately, the meaning of life is a deeply personal and subjective topic that each individual must answer for themselves." 
}

> Finished chain.


{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='What time is it in London?'),
  AIMessage(content='The current time in London, United Kingdom is 8:37:42 PM on November 19, 2023. London is not currently observing Daylight Saving Time (DST).'),
  HumanMessage(content='Can you give me a random number?'),
  AIMessage(content='The response to your last comment was a random number, and the number was 0.')],
 'output': 'The meaning of life is a philosophical and existential question that has been asked by humans for centuries. There is no one definitive answer to this question, and opinions on the topic vary widely depending on cultural, religious, and individual beliefs. Some people believe that the meaning of life is to seek happiness and fulfillment, while others believe it is to achieve spiritual enlightenment or to serve a higher purpose. Ultimately, the meaning of life is a deeply personal and subjective topic that each individual must answer for themse

### for meaning of life it doesn't provide the answer as per in the tool. since that we should change the prompt

In [20]:
print(conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of task

In [14]:
fixed_prompt = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.'''

In [15]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

### here we added **Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.** in the prompt as now it will use the custom tool for the question related to the meaning of life

In [16]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...
{
    "action": "Meaning of Life",
    "action_input": "MOL"
}
Observation: The meaning of life is 42 if rounded but is actually 42.17658
Thought:{
    "action": "Final Answer",
    "action_input": "The meaning of life is 42 if rounded but is actually 42.17658"
}

> Finished chain.


{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='What time is it in London?'),
  AIMessage(content='The current time in London, United Kingdom is 8:37:42 PM on November 19, 2023. London is not currently observing Daylight Saving Time (DST).'),
  HumanMessage(content='Can you give me a random number?'),
  AIMessage(content='The response to your last comment was a random number, and the number was 0.'),
  HumanMessage(content='What is the meaning of life?'),
  AIMessage(content='The meaning of life is a philosophical and existential question that has been asked by humans for centuries. There is no one definitive answer to this question, and opinions on the topic vary widely depending on cultural, religious, and individual beliefs. Some people believe that the meaning of life is to seek happiness and fulfillment, while others believe it is to achieve spiritual enlightenment or to serve a higher purpose. Ultimately, the meaning of life is a deeply personal 

# Let's make a useful one

In [22]:
def stripped_webpage(webpage):
    response = requests.get(webpage)
    html_content = response.text

    def strip_html_tags(html_content):
        soup = BeautifulSoup(html_content, "html.parser")
        stripped_text = soup.get_text()
        return stripped_text

    stripped_content = strip_html_tags(html_content)

    if len(stripped_content) > 4000:
        stripped_content = stripped_content[:4000]
    return stripped_content

In [23]:
stripped_webpage('https://www.google.com')

'GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in\xa0Advanced searchGoogle offered in:  සිංහල தமிழ் AdvertisingAbout GoogleGoogle.lk© 2023 - Privacy - Terms  '

In [24]:
class WebPageTool(BaseTool):
    name = "Get Webpage"
    description = "Useful for when you need to get the content from a specific webpage"

    def _run(self, webpage: str):
        response = requests.get(webpage)
        html_content = response.text

        def strip_html_tags(html_content):
            soup = BeautifulSoup(html_content, "html.parser")
            stripped_text = soup.get_text()
            return stripped_text

        stripped_content = strip_html_tags(html_content)
        if len(stripped_content) > 4000:
            stripped_content = stripped_content[:4000] # avoid max token error
        return stripped_content
    
    def _arun(self, webpage: str):
        raise NotImplementedError("This tool does not support async")

page_getter = WebPageTool()

In [13]:
fixed_prompt = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant also doesn't know information about content on webpages and should always check if asked.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.'''

### //Assistant also doesn't know information about content on webpages and should always check if asked.// 
### This line is what chnaged initial prompt from fixed prompt. it always checks if asked, not just provided by LLM

In [26]:
tools = [page_getter, random_tool, life_tool]

conversational_agent = initialize_agent(
                                        agent='chat-conversational-react-description', 
                                        tools=tools, 
                                        llm=llm,
                                        verbose=True,
                                        max_iterations=3,
                                        memory=memory
                                        )

In [27]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [28]:
conversational_agent.run("Is there an article about Clubhouse on https://techcrunch.com/? today")



> Entering new AgentExecutor chain...
{
    "action": "Get Webpage",
    "action_input": "https://techcrunch.com/search/Clubhouse"
}
Observation: 











 


You searched for clubhouse | TechCrunch























































 


 















TechCrunch


















plus-bold














TechCrunch


Open Navigation






TechCrunch




























				Clubhouse is trying to make a comeback			




		Sep 06, 2023	

Amanda Silberling 




		Remember Clubhouse? At the peak of pandemic lockdowns, the invite-only social audio app soared with 10x month to month growth, then faded from public discourse almost as quickly as a16z investors open	


















				The impact of hype with Clubhouse’s Paul Davison			




		Dec 14, 2022	

Theresa Loconsolo 




		Hello and welcome back to Equity, a podcast about the business of startups, where we unpack the numbers and nuance behind the headlines. This is our Wednesday show, where we niche

"Yes, there is an article about Clubhouse on https://techcrunch.com/ as of November 19, 2023. The article talks about Clubhouse's attempt to make a comeback after fading from public discourse. The article also highlights the impact of hype in the growth of Clubhouse and the recent layoffs done by the company."

In [29]:
conversational_agent.run("What are the titles of the top stories on www.cbsnews.com/?")



> Entering new AgentExecutor chain...
{
    "action": "Get Webpage",
    "action_input": "https://www.cbsnews.com/"
}
Observation: 
CBS News - Breaking news, 24/7 live streaming news & top stories













Israel-Hamas Hostage Deal




Bradley Cooper




Oil Spill In Gulf




Packing Thanksgiving Food




CBS News Live




Managing Your Money




Newsletters


















News




US




World




Politics




Entertainment




HealthWatch




MoneyWatch




CBS Village




Technology




Science




Crime




Sports




Essentials








Israeli government approves deal for hostage release, short cease-fire




Several U.S. service members injured in missile attack at Iraq airbase




OpenAI says Sam Altman returning as CEO days after board sacked him




Ex-Trump Org executive chokes up on stand at fraud trial




Pilot killed when plane crashes in Texas shopping center parking lot




Bob Vander Plaats, influential Iowa evangelical leader, endorses DeSantis




Republican

'The top stories on www.cbsnews.com are: Israeli government approves deal for hostage release, short cease-fire, Several U.S. service members injured in missile attack at Iraq airbase, OpenAI says Sam Altman returning as CEO days after board sacked him, Ex-Trump Org executive chokes up on stand at fraud trial, and Pilot killed when plane crashes in Texas shopping center parking lot.'